In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras

from keras.models import Sequential
from keras.layers import *

from sklearn.model_selection import train_test_split

2023-11-06 11:14:40.213974: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
%run Enzymes_data_NoPrint.ipynb

In [3]:
def letters_to_classes(dic):
    tab = []
    erreur = False
    sequences = [sequ for sequ in dic.values()]
    for i in range(len(sequences)):
        #print(sequences[i])
        tab_lettres = []
        
        for lettre in sequences[i]:
            if lettre == "-":
                tab_lettres.append(0)
            elif (ord(lettre) <= 90) &  (ord(lettre) >= 65):
                tab_lettres.append(ord(lettre)-64)
            else:
                print("Séquence",i,"possède une valeur non standard")
                erreur = True
        if not erreur:
            tab.append(tab_lettres)
            erreur = False
    #print(tab)
    return np.array(tab)           
            
            

In [4]:
dic = {
    "seq1" : "AAGATGATB-TABBAGT",
    "seq2" : "AAGATGATAGAATGAGT",
    "seq3" : "AATATGTAGATATGGAG",
    "seq4" : "TBATABTATTATATABD"
}


non = "XBZJOU"
longueur = 546

In [5]:
letters_to_classes(dic)

array([[ 1,  1,  7,  1, 20,  7,  1, 20,  2,  0, 20,  1,  2,  2,  1,  7,
        20],
       [ 1,  1,  7,  1, 20,  7,  1, 20,  1,  7,  1,  1, 20,  7,  1,  7,
        20],
       [ 1,  1, 20,  1, 20,  7, 20,  1,  7,  1, 20,  1, 20,  7,  7,  1,
         7],
       [20,  2,  1, 20,  1,  2, 20,  1, 20, 20,  1, 20,  1, 20,  1,  2,
         4]])

In [6]:
def get_labels():
    tab = np.random.uniform(0, 1, size=(4,10))
    return np.array(tab)

In [7]:
sequences = get_aligned_sequences()
X = np.array(letters_to_classes(sequences)).astype('float32')
y, dic_solvants, dic_enzymes = get_targets()

#X = X[:100] # Les x premiers
#y = y[:100]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape :", X_train.shape)
print("y_train shape :", y_train.shape)

X_train shape : (207, 546)
y_train shape : (207, 18)


In [8]:
model = Sequential()
model.add(Dense(2000, activation="relu", input_shape=(546,)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1500, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1000, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(500, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(200, activation="relu"))
model.add(BatchNormalization())
model.add(Dense(50, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(18))

2023-11-06 11:14:46.645344: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [9]:
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
#history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2000)              1094000   
                                                                 
 batch_normalization (BatchN  (None, 2000)             8000      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 2000)              0         
                                                                 
 dense_1 (Dense)             (None, 1500)              3001500   
                                                                 
 batch_normalization_1 (Batc  (None, 1500)             6000      
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 1500)              0

In [10]:
pred = model.predict(X)
print("Prediction shape :", pred.shape)
print()
enzyme_index = 0
print("************************** Résultats pour l'enzyme :", dic_enzymes[enzyme_index], "*************************")
print()
for i in range(len(pred[0])):
    print("Pred:",pred[enzyme_index][i],"Target:", y[enzyme_index][i])
#print("Prédiction :",pred[0])
#print("Attendu :", y[0])

9/9 [==============================] - 1s 72ms/step
Prediction shape : (259, 18)

************************** Résultats pour l'enzyme : C7RAM1 *************************

Pred: 0.5885663 Target: 0.8
Pred: 2.1381335 Target: 0.7
Pred: -0.9664899 Target: 0.03
Pred: 0.7953781 Target: 0.1
Pred: -2.512067 Target: 0.12
Pred: 1.4268326 Target: 0.02
Pred: 2.8442063 Target: 0.02
Pred: 4.4371505 Target: 0.01
Pred: -5.404073 Target: 0.06
Pred: 0.6798948 Target: 0.0
Pred: 2.450664 Target: 0.02
Pred: -4.6906595 Target: 0.01
Pred: 0.9550227 Target: 0.01
Pred: -4.4362288 Target: 0.09
Pred: 4.8860397 Target: 0.36
Pred: 6.495889 Target: 0.01
Pred: 0.43481728 Target: 0.01
Pred: -2.6496408 Target: 0.01


In [11]:
courbe = False
if courbe:
    plt.plot(history.history['loss'])
    plt.show()
    plt.plot(history.history['val_loss'])
    plt.show()

In [12]:
# 1D Convolutional Neural Network

In [13]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [14]:
sequences = get_aligned_sequences()
#X = np.array(letters_to_classes(sequences)).astype('float32')
X = np.array(letters_to_classes(sequences))
y, dic_solvants, dic_enzymes = get_targets()
#for i in range(len(X)):
#    print(max(X[i]))
#print(X[0])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
#sequ = ['AABABABBBATBATBATBATB']
#tokenizer = Tokenizer(char_level=True)
#tokenizer.fit_on_texts(sequ)
#sequences = tokenizer.texts_to_sequences(sequ)

#x = pad_sequences(sequences, padding='post')
#print(x[0][4])

In [17]:
model = Sequential()
model.add(Embedding(input_dim=20, output_dim=50, input_length=546))
print(model.output_shape)


(None, 546, 50)


In [18]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 546, 50)           1000      
                                                                 
Total params: 1,000
Trainable params: 1,000
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
model = Sequential()
model.add(Embedding(input_dim=26, output_dim=50, input_length=546))
model.add(Conv1D(filters=32, kernel_size=50, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=20, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=10, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=256, kernel_size=4, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=512, kernel_size=4, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(GlobalMaxPooling1D())
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(18))

In [20]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 546, 50)           1300      
                                                                 
 conv1d (Conv1D)             (None, 497, 32)           80032     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 248, 32)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 229, 64)           41024     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 114, 64)          0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 105, 128)         

In [21]:
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
model.fit(X_train,y_train, validation_data=(X_test, y_test), epochs=60)

Epoch 1/60
7/7 [==============================] - 13s 1s/step - loss: 0.1937 - mse: 0.1937 - val_loss: 0.2883 - val_mse: 0.2883
Epoch 2/60
7/7 [==============================] - 9s 1s/step - loss: 0.1877 - mse: 0.1877 - val_loss: 0.2767 - val_mse: 0.2767
Epoch 3/60
7/7 [==============================] - 10s 1s/step - loss: 0.1779 - mse: 0.1779 - val_loss: 0.2699 - val_mse: 0.2699
Epoch 4/60
7/7 [==============================] - 10s 1s/step - loss: 0.1727 - mse: 0.1727 - val_loss: 0.2665 - val_mse: 0.2665
Epoch 5/60
7/7 [==============================] - 9s 1s/step - loss: 0.1641 - mse: 0.1641 - val_loss: 0.2650 - val_mse: 0.2650
Epoch 6/60
7/7 [==============================] - 10s 1s/step - loss: 0.1558 - mse: 0.1558 - val_loss: 0.2685 - val_mse: 0.2685
Epoch 7/60
7/7 [==============================] - 10s 1s/step - loss: 0.1303 - mse: 0.1303 - val_loss: 0.2686 - val_mse: 0.2686
Epoch 8/60
7/7 [==============================] - 10s 1s/step - loss: 0.0920 - mse: 0.0920 - val_loss: 0.2

KeyboardInterrupt: 